# InSAR Processing Using ASF HYP3 on-demand products

## Getting Started

The Alaska Satellite Facility (ASF) archives SAR and InSAR data on behalf of NASA. 
By searching on their [Vertex website](https://search.asf.alaska.edu/#/?maxResults=250), users can access and download the original data, or in some cases can run basic processing. 
Vertex also allows users to do basic SBAS processing, which produces interferograms that can be downloaded and processed to time-series. 
This is what we will in this exercise. 


The ASF User guide can be found [here](https://docs.asf.alaska.edu/vertex/manual/)

### Steps
The first steps are to download and unzip the products in our working directory. In this case, I have already done this step. 

In [3]:
!ls

S1AA_20220102T235621_20220114T235621_VVP012_INT80_G_ueF_1D0A
S1AA_20220102T235621_20220114T235621_VVP012_INT80_G_ueF_4B23
S1AA_20220102T235621_20220126T235620_VVP024_INT80_G_ueF_6EDA
S1AA_20220102T235621_20220126T235620_VVP024_INT80_G_ueF_AB2C
S1AA_20220102T235621_20220207T235620_VVP036_INT80_G_ueF_3FC2
S1AA_20220102T235621_20220207T235620_VVP036_INT80_G_ueF_8333
S1AA_20220114T235621_20220126T235620_VVP012_INT80_G_ueF_9B8B
S1AA_20220114T235621_20220126T235620_VVP012_INT80_G_ueF_CE01
S1AA_20220114T235621_20220207T235620_VVP024_INT80_G_ueF_E034
S1AA_20220114T235621_20220207T235620_VVP024_INT80_G_ueF_FE45
S1AA_20220114T235621_20220219T235620_VVP036_INT80_G_ueF_799B
S1AA_20220114T235621_20220219T235620_VVP036_INT80_G_ueF_D523
S1AA_20220126T235620_20220207T235620_VVP012_INT80_G_ueF_4080
S1AA_20220126T235620_20220207T235620_VVP012_INT80_G_ueF_ACFA
S1AA_20220126T235620_20220219T235620_VVP024_INT80_G_ueF_2A9C
S1AA_20220126T235620_20220219T235620_VVP024_INT80_G_ueF_BF7C
S1AA_20220126T235620_202

As you can see, we have a number of folders that each contain a set of files, including unwrapped phase, unwrapped displacements, DEM, look vectors, etc. 

Each folder is named by the two dates that were used to generate that interferogram, so for example `S1AA_20220420T235621_20220514T235622_VVP024_INT80_G_ueF_CF44` is an interferogram for dates 4/20/2022 and 5/14/2024. 

In [8]:
# Count the number of interferograms we have downloaded
# If this line does not work on your machine feel free to skip
!ls -ld */ | wc -l  # this will count folders only

     294


The next step is to make sure that all the interferograms have the same shape and geographic extent. 
We will ensure this using a simple code that uses rasterio to re-project all the interferograms to match a reference. 
For this code, you will need `rasterio` and `numpy` installed. 

In [1]:
from process_data import *

In [13]:
# The function to resize the images is called run_resampling
run_resampling(os.getcwd()) # this runs the program on the current directory

Once the resampling has finished, we can now proceed with time-series analysis as normal!
You can refer to the [InSAR lab](https://github.com/jlmaurer/GE6146/blob/master/notebooks/Lab4_InSAR.ipynb) for more details and plotting examples. 

In [2]:
# We'll use the same functions as for the lab
from makeTS import *

In [3]:
ifgList = glob.glob('*/*_unw_phase.tif')
cohList = glob.glob('*/*_corr.tif')

In [4]:
print('I found {} interferograms'.format(len(ifgList)))

I found 294 interferograms


In [5]:
# Pull all the data
datePairs, dates = getDates(ifgList)
fracDates = np.array([dt2fracYear(d) for d in dates])
G = makeG(dates, datePairs)
xSize, ySize, dType, geoProj, trans, noDataVal, Nbands = readRaster(ifgList[0])
data = getData(ifgList,1)

Using band one for dataType


In [6]:
print('Interferograms are {}x{} pixels in size'.format(ySize, xSize))

Interferograms are 2841x3557 pixels in size


In [ ]:
# You'll need to make a plot of one of the dates or the average coherence so you can pick a reference point

In [7]:
# de-referencing
data = dereference(data, taxis=0,refCenter=[2500,2000],refSize=10)

In [8]:
# create the time-series data cube
tsArray = makeTS(G, data, fracDates)

In [9]:
# Convert to displacement in meters
tsArray = convertRad2meters(tsArray, lam=0.056) # wavelength lam is 5.6 cm for Sentinel-1

In [10]:
# Compute the mean velocity by fitting a line
vel = findMeanVel(tsArray, fracDates, 0)

In [11]:
vel.shape

(2841, 3557)